In [1]:
#  w:n:c
#  train : 455(126:105:224)
#  validation : 65(18:15:32)
#  test : 130(36:30:64)
#  총 650

In [240]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.misc import imread
import numpy as np
import os
import tensorflow as tf
import glob
import random
import shutil

IMG_width = 320
IMG_height = 240
IMG_channel = 1
DATASET_PATH = "./dataset_jpg_gray"
kinds = ["train", "validation", "test"]
train_img = np.array([], dtype="float32")
train_label = np.array([], dtype="int8")
validation_img = np.array([], dtype="float32")
validation_label = np.array([], dtype="int8")
test_img = np.array([], dtype="float32")
test_label = np.array([], dtype="int8")
(label_train, filename_train) = (0, 0)
(label_validation, filename_validation) = (0, 0)
(label_test, filename_test) = (0, 0)


def file_info(category_name, dataset_path=DATASET_PATH):
    # 디렉토리 상의 파일경로와 파일의 제일 앞에 매겨진 숫자정보(라벨 정보)를 긁어서 반환
    full_path =  dataset_path + '/' + category_name + '/' + '*.jpg'
    image_filenames = glob.glob(full_path)
    filename = []
    label = []
    for image_filename in image_filenames:
        filename.append(image_filename.split("/")[3])
        label.append(image_filename.split("/")[3].split("-")[0])
    return (label, filename)


def display_image(image, label):
    %matplotlib inline

    fig = plt.figure(figsize=(20, 5))
    for i in range(5):
        print(label[i])
        img = image[i]
        img = img.reshape(240, 320)
        img.shape = (240, 320)
        plt.subplot(150 + (i + 1))
        plt.imshow(img)


def data_processing_about_train(idx, kind):
    for i in range(idx):
        global train_img
        global train_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        train_img = np.append(train_img, image)
        label = _change_one_hot_label(label)
        train_label = np.append(train_label, label)
        # file move
        # 학습한 데이터는 이동시킴
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def data_processing_about_validation(idx, kind):
    for i in range(idx):
        global validation_img
        global validation_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        validation_img = np.append(validation_img, image)
        label = _change_one_hot_label(label)
        validation_label = np.append(validation_label, label)
        # file move
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def data_processing_about_test(idx, kind):
    for i in range(idx):
        global test_img
        global test_label

        (labels, filenames) = file_info(kind)
        random_value = random.randrange(0, len(filenames))
        filename = filenames[random_value]
        label = labels[random_value]
        #         label = _change_one_hot_label(train_label, label)
        #         label = labels[random_value]
        #         image_display(kinds, filename)
        full_path = ( DATASET_PATH + '/' + kind + '/' + filename)
        image = imread(full_path)
        image = image.reshape(IMG_width * IMG_height * IMG_channel) / 255.0
        test_img = np.append(test_img, image)
        label = _change_one_hot_label(label)
        test_label = np.append(test_label, label)
        # file move
        print("---------")
        print(len(filenames))
        print(filename)
        src =  DATASET_PATH + '/' + kind + '/'
        dir =  DATASET_PATH + '/' + 'tmp' + '/' + kind + '/'
        shutil.move(src + filename, dir + filename)


def _change_one_hot_label(target_label):
    target_label = int(target_label)
    T = np.zeros((1, 3))
    T[0][target_label] = 1

    return T


def store_dataset(dataset_path=DATASET_PATH):
    global train_img
    global train_label
    global validation_img
    global validation_label
    global test_img
    global test_label

    (_, idx_train) = file_info("train")
    (_, idx_validation) = file_info("validation")
    (_, idx_test) = file_info("test")
    if len(idx_train) != 0 or len(idx_validation) != 0 or len(idx_test) != 0:
        data_processing_about_train(len(idx_train), "train")
        data_processing_about_validation(len(idx_validation), "validation")
        data_processing_about_test(len(idx_test), "test")

        # file로 쓰기
        train_img.tofile(dataset_path + '/' + 'train_img_dataset.txt')
        train_label.tofile(dataset_path + '/' + 'train_label.txt')
        validation_img.tofile(dataset_path + '/' + 'validation_img_dataset.txt')
        validation_label.tofile(dataset_path + '/' + 'validation_label.txt')
        test_img.tofile(dataset_path + '/' + 'test_img_dataset.txt')
        test_label.tofile(dataset_path + '/' + 'test_label.txt')

    else:
        print("[!] Already Finished Generateing Dataset. Please check directory.")





def load_dataset(dataset_path):
    (img_train, label_train) = (0,0)
    (img_validation, label_validation) = (0,0)
    (img_test, label_test) = (0,0)
    filenames = glob.glob(dataset_path + "/" + "*.txt")
    filename = []
    for filename in filenames:
        print(filename, )
        if filename.split("/")[6][-4:] == ".txt": # CAUTION : filename.split("/")[directory level][-4:]
            img_train = np.fromfile(open(dataset_path + "/" + "train_img_dataset.txt", 'rb')).reshape(451,IMG_width * IMG_height * IMG_channel)
            label_train = np.fromfile(open(dataset_path + "/" + "train_label.txt", 'rb')).reshape(451, 3)
            img_validation = np.fromfile(open(dataset_path + "/" + "validation_img_dataset.txt", 'rb')).reshape(65, IMG_width * IMG_height * IMG_channel)
            label_validation = np.fromfile(open(dataset_path + "/" + "validation_label.txt", 'rb')).reshape(65,3)
            img_test = np.fromfile(open(dataset_path + "/" + "test_img_dataset.txt", 'rb')).reshape(130,IMG_width * IMG_height * IMG_channel)
            label_test = np.fromfile(open(dataset_path + "/" + "test_label.txt", 'rb')).reshape(130, 3)
    return ((img_train, label_train), (img_validation, label_validation), (img_test, label_test))

In [225]:
import time
start = time.time()
store_dataset("imdata")
end = time.time()
print("consumed time : ", int(end-start), "second.")

---------
451
1-0170713_081909_51.jpg
---------
450
1-0170713_174904_24.jpg
---------
449
2-075815_66.jpg
---------
448
2-075711_86.jpg
---------
447
2-121112_25.jpg
---------
446
0-0342_47.jpg
---------
445
2-153843_92.jpg
---------
444
2-025713_26.jpg
---------
443
2-190732_61.jpg
---------
442
2-011819_85.jpg
---------
441
2-074550_90.jpg
---------
440
2-121109_75.jpg
---------
439
1-0170715_093032_78.jpg
---------
438
0-4259_38.jpg
---------
437
0-4614_58.jpg
---------
436
1-0170720_110316_24.jpg
---------
435
0-1014_87.jpg
---------
434
1-0170713_175005_26.jpg
---------
433
0-1418_64.jpg
---------
432
1-0170716_094453_73.jpg
---------
431
2-025708_26.jpg
---------
430
2-094450_79.jpg
---------
429
2-115811_79.jpg
---------
428
0-5743_10.jpg
---------
427
0-1017_38.jpg
---------
426
2-090227_86.jpg
---------
425
2-075243_06.jpg
---------
424
2-075804_92.jpg
---------
423
2-011822_41.jpg
---------
422
2-075810_04.jpg
---------
421
1-0170713_175040_79.jpg
---------
420
0-5745_16.jpg


---------
194
0-5941_97.jpg
---------
193
2-040340_20.jpg
---------
192
2-045947_89.jpg
---------
191
2-094501_42.jpg
---------
190
2-094459_04.jpg
---------
189
2-115612_09.jpg
---------
188
2-190528_03.jpg
---------
187
0-1410_24.jpg
---------
186
2-035411_94.jpg
---------
185
0-0426_80.jpg
---------
184
1-0170718_113346_89.jpg
---------
183
2-193510_37.jpg
---------
182
1-0170716_114431_82.jpg
---------
181
2-110354_63.jpg
---------
180
1-0170716_054901_86.jpg
---------
179
1-0170716_114340_14.jpg
---------
178
2-182014_76.jpg
---------
177
1-0170720_115918_25.jpg
---------
176
2-080047_65.jpg
---------
175
2-065848_69.jpg
---------
174
0-0349_07.jpg
---------
173
1-0170719_060325_45.jpg
---------
172
2-193540_52.jpg
---------
171
2-190540_35.jpg
---------
170
0-0542_65.jpg
---------
169
2-075755_49.jpg
---------
168
0-3439_73.jpg
---------
167
1-0170720_115822_79.jpg
---------
166
2-090349_35.jpg
---------
165
2-083052_15.jpg
---------
164
1-0170716_114357_39.jpg
---------
163
2-19

---------
110
2-093524_17.jpg
---------
109
1-0170719_060313_14.jpg
---------
108
0-0059_29.jpg
---------
107
2-071958_78.jpg
---------
106
1-0170718_113343_33.jpg
---------
105
2-091703_57.jpg
---------
104
2-075123_95.jpg
---------
103
1-0170719_060443_77.jpg
---------
102
0-0311_11.jpg
---------
101
2-090516_81.jpg
---------
100
0-1027_49.jpg
---------
99
2-005134_71.jpg
---------
98
2-200034_95.jpg
---------
97
0-1128_54.jpg
---------
96
0-5619_57.jpg
---------
95
2-012131_01.jpg
---------
94
1-0170720_134436_31.jpg
---------
93
1-0170717_153804_54.jpg
---------
92
1-0170719_060512_50.jpg
---------
91
0-5730_07.jpg
---------
90
2-112542_75.jpg
---------
89
0-5624_64.jpg
---------
88
2-112834_55.jpg
---------
87
2-005011_96.jpg
---------
86
2-091644_07.jpg
---------
85
2-005005_78.jpg
---------
84
2-112742_30.jpg
---------
83
2-101715_99.jpg
---------
82
1-0170716_114422_82.jpg
---------
81
1-0170720_162938_63.jpg
---------
80
2-093503_43.jpg
---------
79
2-223445_50.jpg
---------
7

In [226]:
store_dataset("imdata")

[!] Already Finished Generateing Dataset. Please check directory.


In [227]:
load_dataset("/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata")

/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_label.txt


((array([[ 0.49019608,  0.57647059,  0.59215686, ...,  0.10196078,
           0.10196078,  0.10196078],
         [ 0.75686275,  0.75686275,  0.75294118, ...,  0.65490196,
           0.66666667,  0.64705882],
         [ 0.39607843,  0.39607843,  0.39607843, ...,  0.00392157,
           0.0627451 ,  0.14901961],
         ..., 
         [ 0.68235294,  0.68235294,  0.68235294, ...,  0.38039216,
           0.38039216,  0.38431373],
         [ 0.75686275,  0.75686275,  0.75294118, ...,  0.65882353,
           0.6627451 ,  0.63921569],
         [ 0.48627451,  0.48235294,  0.47058824, ...,  0.48235294,
           0.48235294,  0.48235294]]), array([[ 0.,  1.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.],
         ..., 
         [ 0.,  1.,  0.],
         [ 0.,  1.,  0.],
         [ 1.,  0.,  0.]])),
 (array([[ 0.38823529,  0.39215686,  0.39215686, ...,  0.24313725,
           0.24705882,  0.25098039],
         [ 0.41960784,  0.42352941,  0.42745098, ...,  0.05882353,
           0.058

In [228]:
(img_train, label_train), (img_validation, label_validation), (img_test, label_test) \
                                = load_dataset("/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata")
print(img_train.shape)
print(label_train.shape)
print(img_validation.shape)
print(label_validation.shape)
print(img_test.shape)
print(label_test.shape)

/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_label.txt
(451, 76800)
(451, 3)
(65, 76800)
(65, 3)
(130, 76800)
(130, 3)


In [241]:
display_image(img_train, label_train)

TypeError: 'int' object is not subscriptable

In [236]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))   

def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    return y

def init_network():
    network = {}
    network['W1'] = np.zeros([230400, 128])
    network['b1'] = np.zeros([128])
    network['W2'] = np.zeros([128, 3])
    network['b2'] = np.zeros([3])
    return network
    
def predict(network, x):
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

In [237]:
_, _, (img_test, label_test) = load_dataset("/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata")
network = init_network()
accuracy_cnt = 0

for i in range(len(img_test)):
    y = predict(network, img_test[i])
    p = np.argmax(y, axis=0)
    if p == label_test[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(img_test)))

/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/test_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/train_label.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_img_dataset.txt
/Users/do-hyungkwon/GoogleDrive/jupyter_notebook/imdata/validation_label.txt


ValueError: shapes (76800,) and (230400,128) not aligned: 76800 (dim 0) != 230400 (dim 0)

In [183]:
i = 2
print(img_test[i].shape)

(230400,)


In [186]:
network['W1'].shape

(230400, 128)

In [191]:
a = np.dot(img_test[i], network['W1']) + network['b1']

In [193]:
b = softmax(a)

In [217]:
print(b.shape)

(128,)


In [214]:
p = np.argmax(b, axis=0)

In [215]:
print(p)

0


In [198]:
if (p == label_test[i]):
    print("yes")

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
p = np.argmax(b, axis=0)